In [ ]:
import os
import tensorflow
import numpy as np
import pandas as pd
import math

import keras
import random
import yfinance as yf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.layers import Activation, Flatten, Conv1D, MaxPooling1D

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LeakyReLU

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
from datetime import timedelta
from datetime import date
from datetime import datetime

In [ ]:
data=yf.download(tickers='BTC-USD',end='2023-05-19')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.to_csv('livebitcoindata.csv')

In [ ]:
data=pd.read_csv('livebitcoindata.csv') #loading data

In [ ]:
print(data.shape)
print('Number of days in dataset:',data.shape[0])

(3166, 7)
Number of days in dataset: 3166


In [ ]:
print(data.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [ ]:
print(data.dtypes)

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


In [ ]:
data.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [ ]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3161,2023-05-14,26788.974609,27150.976562,26661.355469,26930.638672,26930.638672,10014858959
3162,2023-05-15,26931.384766,27646.347656,26766.097656,27192.693359,27192.693359,14413231792
3163,2023-05-16,27171.513672,27299.304688,26878.947266,27036.650391,27036.650391,12732238816
3164,2023-05-17,27035.470703,27465.927734,26600.144531,27398.802734,27398.802734,15140006925
3165,2023-05-18,27401.650391,27466.527344,26415.101562,26832.208984,26832.208984,15222938600


In [ ]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3166.000000,3166.000000,3166.000000,3166.000000,3166.000000,3.166000e+03
mean,13376.469719,13707.257676,13012.362089,13383.386779,13383.386779,1.660812e+10
std,16037.521752,16442.814151,15572.496572,16034.362788,16034.362788,1.966574e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,737.157486,746.037750,730.653763,739.508240,739.508240,1.204718e+08
50%,7461.568604,7618.892578,7311.557129,7464.982910,7464.982910,9.653782e+09
75%,19373.141113,19672.721680,19029.088867,19405.889160,19405.889160,2.782872e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,67566.828125,3.509679e+11


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3166 entries, 0 to 3165
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3166 non-null   object 
 1   Open       3166 non-null   float64
 2   High       3166 non-null   float64
 3   Low        3166 non-null   float64
 4   Close      3166 non-null   float64
 5   Adj Close  3166 non-null   float64
 6   Volume     3166 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 173.3+ KB


In [ ]:
nullval=data.isnull().values.sum()
print('Number of null values present:',nullval) #check for any null values
if(nullval>0):
  data=data.interpolate(method="linear") #dealing with null values if any are present
  print("null values dealt with")

Number of null values present: 0


In [ ]:
print('Starting Date: ',data.iloc[0][0])
print('Ending Date: ',data.iloc[-1][0])

Starting Date:  2014-09-17
Ending Date:  2023-05-18


In [ ]:

newdf = data

newdf.drop(newdf[['Adj Close','Volume']],axis=1) #remove unnecessary columns

,Date,Open,High,Low,Close
0,2014-09-17,465.864014,468.174011,452.421997,457.334015
1,2014-09-18,456.859985,456.859985,413.104004,424.440002
2,2014-09-19,424.102997,427.834991,384.532013,394.795990
3,2014-09-20,394.673004,423.295990,389.882996,408.903992
4,2014-09-21,408.084991,412.425995,393.181000,398.821014
...,...,...,...,...,...
3161,2023-05-14,26788.974609,27150.976562,26661.355469,26930.638672
3162,2023-05-15,26931.384766,27646.347656,26766.097656,27192.693359
3163,2023-05-16,27171.513672,27299.304688,26878.947266,27036.650391
3164,2023-05-17,27035.470703,27465.927734,26600.144531,27398.802734


In [ ]:
plot = px.line(newdf, x=newdf.Date, y=[newdf['Open'], newdf['Close'], 
                                          newdf['High'], newdf['Low']],
             labels={'Date': 'Date','value':'Bitcoin Prices'})

plot.show()

In [ ]:
close_data = data[['Date','Close']] #only close price is needed
print(close_data.shape)

(3166, 2)


In [ ]:
close_data.tail()

,Date,Close
3161,2023-05-14,26930.638672
3162,2023-05-15,27192.693359
3163,2023-05-16,27036.650391
3164,2023-05-17,27398.802734
3165,2023-05-18,26832.208984


In [ ]:
close_data.head()

,Date,Close
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014


In [ ]:
df= close_data 
df

,Date,Close
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014
...,...,...
3161,2023-05-14,26930.638672
3162,2023-05-15,27192.693359
3163,2023-05-16,27036.650391
3164,2023-05-17,27398.802734


In [ ]:
scaler = MinMaxScaler()

df = df['Close'].values.reshape(-1, 1)

df = scaler.fit_transform(df) #performing normalization of data


In [ ]:
df.shape

(3166, 1)

In [ ]:
split=int(len(df)*0.80)
train=df[0:split] #splitting training and testing data
test=df[split:len(df),:1]
print("Training data: ", train.shape)
print("Testing data: ", test.shape)

Training data:  (2532, 1)
Testing data:  (634, 1)


In [ ]:
def transform(data, step): #transform into sequences
    x= []
    y= []
    for i in range(len(data)-step-1):  
        x.append(data[i:(i+step), 0] )
        y.append(data[i + step, 0])
    return np.array(x), np.array(y)

In [ ]:
step=20 #window size
xtraining, ytraining =transform(train, step)
xtesting, ytesting = transform(test, step)

print("x training data: ", xtraining.shape)
print("y training data: ", ytraining.shape)
print("x testing data: ", xtesting.shape)
print("y testing data", ytesting.shape)

x training data:  (2511, 20)
y training data:  (2511,)
x testing data:  (613, 20)
y testing data (613,)


In [ ]:
xtraining =xtraining.reshape(xtraining.shape[0],xtraining.shape[1] , 1) #reshape the data into required shape for input to model
xtesting = xtesting.reshape(xtesting.shape[0],xtesting.shape[1] , 1)

print("X training data: ", xtraining.shape)
print("X testing data: ", xtesting.shape)

X training data:  (2511, 20, 1)
X testing data:  (613, 20, 1)


In [ ]:

ytraining=ytraining.reshape(ytraining.shape[0],1)
ytesting=ytesting.reshape(ytesting.shape[0],1)
print("Y training data: ",ytraining.shape)
print("Y testing data: ",ytesting.shape)

Y training data:  (2511, 1)
Y testing data:  (613, 1)


In [ ]:
dates=close_data['Date'] 
dates=dates[int(len(df)*0.80)+step:-1] #separating the dates for test data
print(dates)

2552    2021-09-12
2553    2021-09-13
2554    2021-09-14
2555    2021-09-15
2556    2021-09-16
           ...    
3160    2023-05-13
3161    2023-05-14
3162    2023-05-15
3163    2023-05-16
3164    2023-05-17
Name: Date, Length: 613, dtype: object


In [ ]:
y_inverse = scaler.inverse_transform(ytesting)
y_inverse= y_inverse.reshape(y_inverse.shape[0])

In [ ]:
def modelpred(model): #predicting using the model

  ypredict = model.predict(xtesting)
  yp_inverse = scaler.inverse_transform(ypredict)
  yp_inverse=yp_inverse.reshape(yp_inverse.shape[0])
  return yp_inverse

In [ ]:
def plotseries(dates,actual,predicted): #plot actual price vs predicted price

  plotdf=pd.DataFrame()
  plotdf['Date']=dates
  plotdf['Actual Price']=actual
  plotdf['Predicted Price']=predicted

  plot = px.line(plotdf, x='Date', y=['Actual Price','Predicted Price'],
             labels={'x': 'Date','value':'Bitcoin Prices'})

  plot.show()

CNN WITH 2 LAYERS

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(40)
np.random.seed(40)
random.seed(40)


cnnmodel= Sequential()

cnnmodel.add(Conv1D(64, kernel_size=3, activation='relu',input_shape=(xtraining.shape[1],1)))
cnnmodel.add(MaxPooling1D(pool_size=2))

cnnmodel.add(Conv1D(128, kernel_size=3, activation='relu',padding='same'))
cnnmodel.add(MaxPooling1D(pool_size=2))


cnnmodel.add(Flatten())

cnnmodel.add(Dense(64, activation='relu'))

cnnmodel.add(Dense(1, activation='linear'))

cnnmodel.compile(loss=keras.losses.Huber(), optimizer='adam')
checkpoint = keras.callbacks.ModelCheckpoint("cnn_checkpoint1.h5" ,save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=100)
cnnmodel.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint],shuffle=False)

Epoch 1/300
36/36 [==============================] - 7s 46ms/step - loss: 7.5131e-04 - val_loss: 0.0263
Epoch 2/300
36/36 [==============================] - 1s 22ms/step - loss: 0.0012 - val_loss: 0.0050
Epoch 3/300
36/36 [==============================] - 1s 19ms/step - loss: 4.8252e-04 - val_loss: 0.0045
Epoch 4/300
36/36 [==============================] - 1s 21ms/step - loss: 2.3401e-04 - val_loss: 0.0037
Epoch 5/300
36/36 [==============================] - 1s 20ms/step - loss: 1.4193e-04 - val_loss: 0.0033
Epoch 6/300
36/36 [==============================] - 1s 17ms/step - loss: 1.3097e-04 - val_loss: 0.0032
Epoch 7/300
36/36 [==============================] - 1s 21ms/step - loss: 2.4229e-04 - val_loss: 0.0031
Epoch 8/300
36/36 [==============================] - 1s 20ms/step - loss: 2.6017e-04 - val_loss: 0.0031
Epoch 9/300
36/36 [==============================] - 1s 16ms/step - loss: 3.8646e-04 - val_loss: 0.0033
Epoch 10/300
36/36 [==============================] - 1s 17ms/step -

In [ ]:

loss=min(cnnmodel.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000852


In [ ]:
cnnmodel = keras.models.load_model("cnn_checkpoint1.h5")

In [ ]:
yp1=modelpred(cnnmodel)

20/20 [==============================] - 0s 3ms/step


In [ ]:
plotseries(dates,y_inverse,yp1)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp1).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp1).numpy()

print("RMSE: ",round(math.sqrt(m),4))

mape = np.mean(np.abs((y_inverse - yp1)/y_inverse))*100
print("MAPE: ",round(mape,2))

MAE:  1280.9219
RMSE:  1790.0154
MAPE:  4.09


CNN WITH 3 LAYERS

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(40)
np.random.seed(40)
random.seed(40)



cnnmodel2= Sequential()


cnnmodel2.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(xtraining.shape[1],1)))

cnnmodel2.add(MaxPooling1D(pool_size=2))

cnnmodel2.add(Conv1D(64, kernel_size=3, activation='relu',padding='same'))

cnnmodel2.add(MaxPooling1D(pool_size=2))

cnnmodel2.add(Conv1D(64, kernel_size=2, activation='relu',padding='same'))

cnnmodel2.add(MaxPooling1D(pool_size=2))

cnnmodel2.add(Flatten())

cnnmodel2.add(Dense(64, activation='relu'))

cnnmodel2.add(Dense(1, activation='linear'))

cnnmodel2.compile(loss=keras.losses.Huber(), optimizer='adam')
checkpoint = keras.callbacks.ModelCheckpoint("cnn_checkpoint2.h5", save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=100)
cnnmodel2.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint],shuffle=False)

Epoch 1/300
36/36 [==============================] - 2s 16ms/step - loss: 8.7691e-04 - val_loss: 0.0268
Epoch 2/300
36/36 [==============================] - 0s 9ms/step - loss: 9.9347e-04 - val_loss: 0.0042
Epoch 3/300
36/36 [==============================] - 0s 7ms/step - loss: 7.7536e-04 - val_loss: 0.0046
Epoch 4/300
36/36 [==============================] - 0s 8ms/step - loss: 7.6752e-04 - val_loss: 0.0041
Epoch 5/300
36/36 [==============================] - 0s 8ms/step - loss: 5.9638e-04 - val_loss: 0.0045
Epoch 6/300
36/36 [==============================] - 0s 8ms/step - loss: 9.2164e-04 - val_loss: 0.0118
Epoch 7/300
36/36 [==============================] - 0s 7ms/step - loss: 0.0015 - val_loss: 0.0290
Epoch 8/300
36/36 [==============================] - 0s 8ms/step - loss: 8.3143e-04 - val_loss: 0.0351
Epoch 9/300
36/36 [==============================] - 0s 8ms/step - loss: 3.3609e-04 - val_loss: 0.0277
Epoch 10/300
36/36 [==============================] - 0s 7ms/step - loss: 2.

In [ ]:

loss=min(cnnmodel2.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000902


In [ ]:
cnnmodel2 = keras.models.load_model("cnn_checkpoint2.h5")

In [ ]:
yp2=modelpred(cnnmodel2)

20/20 [==============================] - 0s 2ms/step


In [ ]:
plotseries(dates,y_inverse,yp2)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp2).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp2).numpy()

print("RMSE: ",round(math.sqrt(m),4))


mape = np.mean(np.abs((y_inverse - yp2)/y_inverse))*100
print("MAPE: ",round(mape,2))







MAE:  1249.5969
RMSE:  1855.9951
MAPE:  3.88


LSTM - 3 LAYERS

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(40)
np.random.seed(40)
random.seed(40)

lstm1 = Sequential()
lstm1.add(LSTM(32, return_sequences=True, activation='tanh',
               input_shape=(xtraining.shape[1],1)))



lstm1.add(LSTM(64, return_sequences=True,activation='tanh'))

#lstm1.add(Dropout(0.1))

lstm1.add(LSTM(128,activation='tanh'))


lstm1.add(Dense(1))
lstm1.add(LeakyReLU())


lstm1.compile(loss=keras.losses.Huber(), optimizer='adam')
checkpoint = keras.callbacks.ModelCheckpoint("lstm_checkpoint1.h5", save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=50)
lstm1.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint])

Epoch 1/300
36/36 [==============================] - 12s 132ms/step - loss: 6.5254e-04 - val_loss: 0.0031
Epoch 2/300
36/36 [==============================] - 4s 113ms/step - loss: 8.4270e-05 - val_loss: 0.0031
Epoch 3/300
36/36 [==============================] - 3s 71ms/step - loss: 7.2419e-05 - val_loss: 0.0033
Epoch 4/300
36/36 [==============================] - 3s 73ms/step - loss: 7.7305e-05 - val_loss: 0.0033
Epoch 5/300
36/36 [==============================] - 3s 71ms/step - loss: 6.4277e-05 - val_loss: 0.0034
Epoch 6/300
36/36 [==============================] - 3s 86ms/step - loss: 6.9206e-05 - val_loss: 0.0036
Epoch 7/300
36/36 [==============================] - 3s 96ms/step - loss: 5.1083e-05 - val_loss: 0.0036
Epoch 8/300
36/36 [==============================] - 3s 72ms/step - loss: 5.5083e-05 - val_loss: 0.0025
Epoch 9/300
36/36 [==============================] - 3s 73ms/step - loss: 4.5453e-05 - val_loss: 0.0050
Epoch 10/300
36/36 [==============================] - 3s 73ms

In [ ]:

loss=min(lstm1.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000418


In [ ]:
lstm1 = keras.models.load_model("lstm_checkpoint1.h5")

In [ ]:
yp3=modelpred(lstm1)

20/20 [==============================] - 2s 16ms/step


In [ ]:
plotseries(dates,y_inverse,yp3)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp3).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp3).numpy()

print("RMSE: ",round(math.sqrt(m),4))

mape = np.mean(np.abs((y_inverse - yp3)/y_inverse))*100
print("MAPE: ",round(mape,2))


MAE:  942.1879
RMSE:  1295.5259
MAPE:  3.03


LSTM - 2 LAYERS  

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(42)
np.random.seed(42)
random.seed(42)

lstm2 = Sequential()
lstm2.add(LSTM(64, return_sequences=True, activation='tanh',
               input_shape=(xtraining.shape[1],1)))


lstm2.add(LSTM(128,activation='tanh'))

lstm2.add(Dense(1))
lstm2.add(LeakyReLU())

lstm2.compile(loss=keras.losses.Huber(), optimizer='adam')
checkpoint = keras.callbacks.ModelCheckpoint("lstm_checkpoint2.h5", save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=50)
lstm2.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint])

Epoch 1/300
36/36 [==============================] - 9s 107ms/step - loss: 5.2763e-04 - val_loss: 0.0044
Epoch 2/300
36/36 [==============================] - 3s 87ms/step - loss: 5.0950e-05 - val_loss: 0.0023
Epoch 3/300
36/36 [==============================] - 2s 67ms/step - loss: 4.9472e-05 - val_loss: 0.0018
Epoch 4/300
36/36 [==============================] - 2s 62ms/step - loss: 4.3857e-05 - val_loss: 0.0020
Epoch 5/300
36/36 [==============================] - 2s 64ms/step - loss: 4.0562e-05 - val_loss: 0.0015
Epoch 6/300
36/36 [==============================] - 3s 86ms/step - loss: 3.7465e-05 - val_loss: 0.0019
Epoch 7/300
36/36 [==============================] - 3s 80ms/step - loss: 3.5985e-05 - val_loss: 0.0030
Epoch 8/300
36/36 [==============================] - 2s 68ms/step - loss: 4.0415e-05 - val_loss: 0.0033
Epoch 9/300
36/36 [==============================] - 2s 66ms/step - loss: 3.3223e-05 - val_loss: 0.0022
Epoch 10/300
36/36 [==============================] - 2s 65ms/s

In [ ]:

loss=min(lstm2.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000401


In [ ]:
lstm2 = keras.models.load_model("lstm_checkpoint2.h5")

In [ ]:
yp4=modelpred(lstm2)

20/20 [==============================] - 1s 16ms/step


In [ ]:
plotseries(dates,y_inverse,yp4)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp4).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp4).numpy()

print("RMSE: ",round(math.sqrt(m),4))

mape = np.mean(np.abs((y_inverse - yp4)/y_inverse))*100
print("MAPE: ",round(mape,2))

MAE:  845.64246
RMSE:  1195.4698
MAPE:  2.67


LSTM - 1 LAYER

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(42)
np.random.seed(42)
random.seed(42)

lstm3 = Sequential()
lstm3.add(LSTM(128,activation='tanh', input_shape=(xtraining.shape[1],1),return_sequences=False))

lstm3.add(Dense(1))
lstm3.add(LeakyReLU())
lstm3.compile(loss=keras.losses.Huber(), optimizer='adam')

checkpoint = keras.callbacks.ModelCheckpoint("lstm_checkpoint3.h5", save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=50)
lstm3.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint])




Epoch 1/300
36/36 [==============================] - 4s 57ms/step - loss: 5.9233e-04 - val_loss: 0.0021
Epoch 2/300
36/36 [==============================] - 1s 37ms/step - loss: 4.2838e-05 - val_loss: 0.0017
Epoch 3/300
36/36 [==============================] - 2s 49ms/step - loss: 3.9159e-05 - val_loss: 0.0016
Epoch 4/300
36/36 [==============================] - 3s 74ms/step - loss: 3.6139e-05 - val_loss: 0.0015
Epoch 5/300
36/36 [==============================] - 1s 40ms/step - loss: 3.7705e-05 - val_loss: 0.0014
Epoch 6/300
36/36 [==============================] - 1s 38ms/step - loss: 3.2760e-05 - val_loss: 0.0013
Epoch 7/300
36/36 [==============================] - 1s 39ms/step - loss: 3.2456e-05 - val_loss: 0.0015
Epoch 8/300
36/36 [==============================] - 1s 39ms/step - loss: 3.3002e-05 - val_loss: 0.0015
Epoch 9/300
36/36 [==============================] - 1s 39ms/step - loss: 3.0091e-05 - val_loss: 0.0012
Epoch 10/300
36/36 [==============================] - 1s 40ms/st

In [ ]:

loss=min(lstm3.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000388


In [ ]:
lstm3= keras.models.load_model("lstm_checkpoint3.h5")

In [ ]:
yp5=modelpred(lstm3)

20/20 [==============================] - 2s 17ms/step


In [ ]:
plotseries(dates,y_inverse,yp5)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp5).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp5).numpy()

print("RMSE: ",round(math.sqrt(m),4))

mape = np.mean(np.abs((y_inverse - yp5)/y_inverse))*100
print("MAPE: ",round(mape,2))

MAE:  799.7471
RMSE:  1180.0096
MAPE:  2.48


GRU WITH 3 LAYERS

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(42)
np.random.seed(42)
random.seed(42)


gru1 = Sequential()

gru1.add(GRU(64, return_sequences=True, input_shape=(xtraining.shape[1],1), activation='tanh'))

gru1.add(GRU(64, return_sequences=True, activation='tanh'))

gru1.add(GRU(128, activation='tanh'))

gru1.add(Dense(units=1))

gru1.compile(loss=keras.losses.Huber(), optimizer='adam')

checkpoint = keras.callbacks.ModelCheckpoint("gru_checkpoint1.h5", save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=50)
gru1.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint])

Epoch 1/300
36/36 [==============================] - 13s 172ms/step - loss: 6.2107e-04 - val_loss: 0.0013
Epoch 2/300
36/36 [==============================] - 3s 82ms/step - loss: 3.7300e-05 - val_loss: 0.0011
Epoch 3/300
36/36 [==============================] - 3s 72ms/step - loss: 2.7603e-05 - val_loss: 0.0011
Epoch 4/300
36/36 [==============================] - 3s 79ms/step - loss: 2.5932e-05 - val_loss: 0.0010
Epoch 5/300
36/36 [==============================] - 3s 86ms/step - loss: 2.6709e-05 - val_loss: 9.8713e-04
Epoch 6/300
36/36 [==============================] - 3s 93ms/step - loss: 2.5053e-05 - val_loss: 9.6080e-04
Epoch 7/300
36/36 [==============================] - 3s 77ms/step - loss: 2.3277e-05 - val_loss: 9.5019e-04
Epoch 8/300
36/36 [==============================] - 3s 74ms/step - loss: 2.2095e-05 - val_loss: 9.0187e-04
Epoch 9/300
36/36 [==============================] - 3s 75ms/step - loss: 2.1433e-05 - val_loss: 7.8760e-04
Epoch 10/300
36/36 [======================

In [ ]:

loss=min(gru1.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000389


In [ ]:
gru1= keras.models.load_model("gru_checkpoint1.h5")

In [ ]:
yp6=modelpred(gru1)

20/20 [==============================] - 1s 13ms/step


In [ ]:
plotseries(dates,y_inverse,yp6)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp6).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp6).numpy()

print("RMSE: ",round(math.sqrt(m),4))

mape = np.mean(np.abs((y_inverse - yp6)/y_inverse))*100
print("MAPE: ",round(mape,2))

MAE:  823.5123
RMSE:  1198.7813
MAPE:  2.57


GRU WITH 2 LAYERS

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(42)
np.random.seed(42)
random.seed(42)


gru2 = Sequential()

gru2.add(GRU(64, return_sequences=True, input_shape=(xtraining.shape[1],1), activation='tanh'))

gru2.add(GRU(128, activation='tanh'))

gru2.add(Dense(1))

gru2.compile(loss=keras.losses.Huber(), optimizer='adam')

checkpoint = keras.callbacks.ModelCheckpoint("gru_checkpoint2.h5", save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=50)
gru2.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint])

Epoch 1/300
36/36 [==============================] - 8s 85ms/step - loss: 3.7931e-04 - val_loss: 0.0024
Epoch 2/300
36/36 [==============================] - 3s 94ms/step - loss: 2.7514e-05 - val_loss: 8.8700e-04
Epoch 3/300
36/36 [==============================] - 2s 55ms/step - loss: 2.1847e-05 - val_loss: 8.4642e-04
Epoch 4/300
36/36 [==============================] - 2s 58ms/step - loss: 2.0417e-05 - val_loss: 8.0391e-04
Epoch 5/300
36/36 [==============================] - 2s 53ms/step - loss: 2.0078e-05 - val_loss: 7.8034e-04
Epoch 6/300
36/36 [==============================] - 2s 55ms/step - loss: 1.9192e-05 - val_loss: 7.4799e-04
Epoch 7/300
36/36 [==============================] - 2s 61ms/step - loss: 1.8420e-05 - val_loss: 7.2578e-04
Epoch 8/300
36/36 [==============================] - 3s 85ms/step - loss: 1.9537e-05 - val_loss: 8.0028e-04
Epoch 9/300
36/36 [==============================] - 2s 56ms/step - loss: 1.8867e-05 - val_loss: 6.4335e-04
Epoch 10/300
36/36 [============

In [ ]:

loss=min(gru2.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000390


In [ ]:
gru2= keras.models.load_model("gru_checkpoint2.h5")

In [ ]:
yp7=modelpred(gru2)

20/20 [==============================] - 2s 10ms/step


In [ ]:
plotseries(dates,y_inverse,yp7)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp7).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp7).numpy()

print("RMSE: ",round(math.sqrt(m),4))

mape = np.mean(np.abs((y_inverse - yp7)/y_inverse))*100
print("MAPE: ",round(mape,2))

MAE:  793.90814
RMSE:  1184.2317
MAPE:  2.46


GRU WITH 1 LAYER

In [ ]:
keras.backend.clear_session()
tensorflow.random.set_seed(42)
np.random.seed(42)
random.seed(42)


gru3 = Sequential()

gru3.add(GRU(128, input_shape=(xtraining.shape[1],1), activation='tanh',return_sequences=False))

gru3.add(Dense(units=1))

gru3.compile(loss=keras.losses.Huber(), optimizer='adam')

checkpoint = keras.callbacks.ModelCheckpoint("gru_checkpoint3.h5", save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=50)
gru3.fit(xtraining, ytraining, epochs=300, batch_size=64, validation_split=0.1, callbacks=[earlystop, checkpoint])

Epoch 1/300
36/36 [==============================] - 5s 82ms/step - loss: 8.4077e-04 - val_loss: 0.0016
Epoch 2/300
36/36 [==============================] - 1s 33ms/step - loss: 3.0105e-05 - val_loss: 9.4218e-04
Epoch 3/300
36/36 [==============================] - 1s 36ms/step - loss: 2.2504e-05 - val_loss: 8.9955e-04
Epoch 4/300
36/36 [==============================] - 1s 35ms/step - loss: 2.1707e-05 - val_loss: 8.8499e-04
Epoch 5/300
36/36 [==============================] - 1s 34ms/step - loss: 2.1647e-05 - val_loss: 8.7124e-04
Epoch 6/300
36/36 [==============================] - 1s 36ms/step - loss: 2.0880e-05 - val_loss: 8.2425e-04
Epoch 7/300
36/36 [==============================] - 1s 36ms/step - loss: 2.0494e-05 - val_loss: 7.9879e-04
Epoch 8/300
36/36 [==============================] - 1s 33ms/step - loss: 2.0525e-05 - val_loss: 8.8527e-04
Epoch 9/300
36/36 [==============================] - 2s 43ms/step - loss: 1.9684e-05 - val_loss: 7.4514e-04
Epoch 10/300
36/36 [============

In [ ]:

loss=min(gru3.history.history['val_loss'])
print("Lowest value of validation loss:",f'{loss:.6f}')

Lowest value of validation loss: 0.000390


In [ ]:
gru3 = keras.models.load_model("gru_checkpoint3.h5")

In [ ]:
yp8=modelpred(gru3)

20/20 [==============================] - 1s 7ms/step


In [ ]:
plotseries(dates,y_inverse,yp8)

In [ ]:
print("MAE: ",keras.metrics.mean_absolute_error(y_inverse, yp8).numpy())

m=keras.metrics.mean_squared_error(y_inverse, yp8).numpy()

print("RMSE: ",round(math.sqrt(m),4))

mape = np.mean(np.abs((y_inverse - yp8)/y_inverse))*100
print("MAPE: ",round(mape,2))

MAE:  777.515
RMSE:  1164.1768
MAPE:  2.4


PREDICTION FOR 3 DAYS

In [ ]:
def pred_next(model):

    input=test[len(test)-step:].reshape(1,-1)
    temporary=list(input)
    temporary=temporary[0].tolist()

    output=[]
    i=0
    no_days = 3
    while(i<no_days):
      if(len(temporary)>step):
            
            input=np.array(temporary[1:])

            input = input.reshape(1,-1)
            input = input.reshape((1,step, 1))
            
            prediction = model.predict(input, verbose=0)
            temporary.extend(prediction[0].tolist())
            temporary=temporary[1:]
          
            output.extend(prediction.tolist())
            i=i+1

      else:
            
            input = input.reshape((1,step,1))
            prediction = model.predict(input, verbose=0)
            temporary.extend(prediction[0].tolist())
            
            output.extend(prediction.tolist())
            i=i+1
            
    op= scaler.inverse_transform(output).reshape(1,-1)             
    return(op[0])

In [ ]:
plist=pred_next(gru3) #predicting using GRU with 1 layer
end=dates.iloc[-1]
end=datetime.strptime(end, "%Y-%m-%d").date()
print("Predicted prices for following 3 days:\n")
print("Date\t\tPredicted Price")
for i in range(len(plist)):
 
  print(end+timedelta(i+1),"\t",round(plist[i],2))



Predicted prices for following 3 days:

Date		Predicted Price
2023-05-18 	 27101.8
2023-05-19 	 27260.78
2023-05-20 	 27446.93
